#### Import Lib

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, time, pygame, json, copy

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Load Camera Parameters

In [2]:
with open('./save_folder/camera_parameter_1103.json', 'r') as f:
    camera_parameter = json.load(f)
    
Extrinsic_parameters = camera_parameter['Extrinsic parameters']
Intrinsic_parameters = camera_parameter['Intrinsic parameters']

translation_vector = Extrinsic_parameters['translation_vector']
rotation_matrix = Extrinsic_parameters['rotation_matrix']

Intrinsic_parameters    = np.mat(Intrinsic_parameters, np.float64)
translation_vector      = np.mat(translation_vector, np.double)
rotation_matrix         = rotation_matrix

#### Function

In [3]:
def pixel_to_world(img_points):
    camera_intrinsics = Intrinsic_parameters
    r                 = rotation_matrix
    t                 = translation_vector
    
    K_inv = camera_intrinsics.I
    R_inv = np.asmatrix(r).I
    R_inv_T = np.dot(R_inv, np.asmatrix(t))
    world_points = []
    coords = np.zeros((3, 1), dtype=np.float64)
    for img_point in img_points:
        coords[0] = img_point[0]
        coords[1] = img_point[1]
        coords[2] = 1.0
        cam_point = np.dot(K_inv, coords)
        cam_R_inv = np.dot(R_inv, cam_point)
        scale = R_inv_T[2][0] / cam_R_inv[2][0]
        scale_world = np.multiply(scale, cam_R_inv)
        world_point = np.asmatrix(scale_world) - np.asmatrix(R_inv_T)
        pt = np.zeros((3, 1), dtype=np.float64)
        pt[0] = world_point[0]
        pt[1] = world_point[1]
        pt[2] = 0
        world_points.append(pt.T.tolist())

    return world_points

def make_character(id):
    text = str(id)

    pos = (2, 24) if len(text) == 2 else (9, 24)
    obj = np.full((32, 32, 3), (0, 0, 0), dtype=np.uint8)

    cv2.putText(obj, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    obj = cv2.rotate(obj, cv2.ROTATE_90_COUNTERCLOCKWISE)
    obj = cv2.flip(obj, 0) 
    obj_character = pygame.surfarray.make_surface(obj)
    
    return obj_character

#### Game

In [4]:
pygame.init() #초기화q

screen_width = 480
screen_height = 480
screen = pygame.display.set_mode((screen_width, screen_height))

pygame.display.set_caption("map")

background = pygame.image.load("C:/Users/user/Desktop/Multi-Target-Multi-Camera-Tracking-Capstone-Design-main/Multi-Target-Multi-Camera-Tracking-Capstone-Design-main/2d img xy to 3d world xyz Usage/img_folder/map.png")

def mouse_callback(event, x, y, flags, param):
    cv2.imshow("image", image_origin)
    image = copy.deepcopy(image_origin)

    result = pixel_to_world(np.array(([[x, y]]), dtype=np.double))
    x_w = int(result[0][0][0])
    y_w = int(result[0][0][1])
    gain = 25
    
    cv2.circle(image, (x, y), 6, (0, 0, 255), 3)
    cv2.putText(image, text='({}, {})'.format(x_w, y_w), org=(x, y-10),\
        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0), thickness=2)
    cv2.imshow("image", image)
    
    id = 1
    screen.blit(background, (0, 0))
    screen.blit(make_character(id), (x_w*gain, y_w*gain)) 
    pygame.display.update()

# 이미지 파일 읽기

image_origin = cv2.imread("./img_folder/frame_img.jpg")
image_origin = cv2.resize(image_origin, dsize = (640, 480), interpolation = cv2.INTER_AREA)
cv2.arrowedLine(image_origin, (167, 270), (283, 298), (0, 0, 0), thickness=2) # x축
cv2.arrowedLine(image_origin, (167, 270), (88, 368), (0, 0, 0), thickness=2) # y축
cv2.putText(image_origin, text='x-axis', org=(283+5, 298+5),\
        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 0), thickness=2)
cv2.putText(image_origin, text='y-axis', org=(88+5, 368+5),\
        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 0), thickness=2)

cv2.namedWindow("image")
cv2.setMouseCallback("image", mouse_callback)
cv2.waitKey(0)
cv2.destroyAllWindows()
